# Risk Management Framework Demo

This notebook demonstrates the risk management framework for the copilot_quant platform.

We'll cover:
1. Basic risk settings configuration
2. Portfolio risk checks
3. Position-level risk controls
4. Volatility-based position sizing
5. Circuit breaker mechanism
6. Comparing strategies with different risk profiles

## Capital Preservation Philosophy

All risk controls are designed with one priority: **DON'T LOSE MONEY**

- Conservative defaults limit downside
- Circuit breaker prevents catastrophic losses
- Position limits prevent over-concentration
- Stop losses protect individual positions

In [ ]:
# Import required libraries
import sys
from pathlib import Path

import numpy as np
import pandas as pd

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from risk.portfolio_risk import RiskManager
from risk.settings import RiskSettings

## 1. Risk Settings Configuration

Let's explore the three built-in risk profiles: Conservative, Balanced, and Aggressive.

In [ ]:
# Create three risk profiles
conservative = RiskSettings.get_conservative_profile()
balanced = RiskSettings.get_balanced_profile()
aggressive = RiskSettings.get_aggressive_profile()

# Compare key parameters
comparison = pd.DataFrame({
    'Parameter': ['Max Drawdown', 'Max Position Size', 'Min Cash Buffer', 'Position Stop Loss', 'Max Positions', 'Circuit Breaker', 'Target Volatility'],
    'Conservative': [
        f"{conservative.max_portfolio_drawdown:.0%}",
        f"{conservative.max_position_size:.0%}",
        f"{conservative.min_cash_buffer:.0%}",
        f"{conservative.position_stop_loss:.0%}",
        conservative.max_positions,
        f"{conservative.circuit_breaker_threshold:.0%}",
        f"{conservative.target_portfolio_volatility:.0%}"
    ],
    'Balanced': [
        f"{balanced.max_portfolio_drawdown:.0%}",
        f"{balanced.max_position_size:.0%}",
        f"{balanced.min_cash_buffer:.0%}",
        f"{balanced.position_stop_loss:.0%}",
        balanced.max_positions,
        f"{balanced.circuit_breaker_threshold:.0%}",
        f"{balanced.target_portfolio_volatility:.0%}"
    ],
    'Aggressive': [
        f"{aggressive.max_portfolio_drawdown:.0%}",
        f"{aggressive.max_position_size:.0%}",
        f"{aggressive.min_cash_buffer:.0%}",
        f"{aggressive.position_stop_loss:.0%}",
        aggressive.max_positions,
        f"{aggressive.circuit_breaker_threshold:.0%}",
        f"{aggressive.target_portfolio_volatility:.0%}"
    ]
})

print("\nRisk Profile Comparison:")
print(comparison.to_string(index=False))

## 2. Portfolio Risk Checks

Let's simulate a portfolio and see how risk checks work.

In [ ]:
# Create a risk manager with conservative settings
risk_manager = RiskManager(conservative)

# Simulate portfolio scenarios
scenarios = [
    {"name": "Healthy Portfolio", "value": 95000, "peak": 100000, "cash": 25000},
    {"name": "Low Cash Buffer", "value": 100000, "peak": 100000, "cash": 15000},
    {"name": "High Drawdown", "value": 85000, "peak": 100000, "cash": 25000},
    {"name": "Circuit Breaker Trigger", "value": 90000, "peak": 100000, "cash": 25000},
]

print("\nPortfolio Risk Check Results:\n" + "="*80)
for scenario in scenarios:
    result = risk_manager.check_portfolio_risk(
        portfolio_value=scenario["value"],
        peak_value=scenario["peak"],
        cash=scenario["cash"]
    )
    
    status = "✅ APPROVED" if result.approved else "❌ REJECTED"
    print(f"\n{scenario['name']}:")
    print(f"  Status: {status}")
    print(f"  Reason: {result.reason}")
    if result.details:
        if 'current_drawdown' in result.details:
            print(f"  Drawdown: {result.details['current_drawdown']:.1%}")
        if 'cash_pct' in result.details:
            print(f"  Cash %: {result.details['cash_pct']:.1%}")

## 3. Position-Level Risk Controls

Test position size limits and stop losses.

In [ ]:
# Reset risk manager
risk_manager = RiskManager(conservative)
portfolio_value = 100000

# Test various position scenarios
position_scenarios = [
    {"name": "Normal Position", "size": 8000, "entry": 100, "current": 102},
    {"name": "Oversized Position", "size": 12000, "entry": 100, "current": 102},
    {"name": "Stop Loss Triggered", "size": 8000, "entry": 100, "current": 92},
    {"name": "Winning Position", "size": 8000, "entry": 100, "current": 110},
]

print("\nPosition Risk Check Results:\n" + "="*80)
for scenario in position_scenarios:
    result = risk_manager.check_position_risk(
        position_value=scenario["size"],
        portfolio_value=portfolio_value,
        entry_price=scenario["entry"],
        current_price=scenario["current"],
        symbol="TEST"
    )
    
    status = "✅ APPROVED" if result.approved else "❌ REJECTED"
    position_return = (scenario["current"] - scenario["entry"]) / scenario["entry"]
    
    print(f"\n{scenario['name']}:")
    print(f"  Status: {status}")
    print(f"  Position Size: ${scenario['size']:,} ({scenario['size']/portfolio_value:.1%} of portfolio)")
    print(f"  Return: {position_return:.1%}")
    print(f"  Reason: {result.reason}")

## 4. Volatility-Based Position Sizing

Demonstrate how volatility targeting adjusts position sizes.

In [ ]:
# Test position sizing with different volatilities
risk_manager = RiskManager(conservative)
portfolio_value = 100000
signal_strength = 1.0  # Full conviction

# Test various volatility levels
volatilities = [0.10, 0.15, 0.20, 0.30, 0.40, 0.50]
position_sizes = []

for vol in volatilities:
    size = risk_manager.calculate_position_size(
        signal_strength=signal_strength,
        portfolio_value=portfolio_value,
        volatility=vol
    )
    position_sizes.append(size)

# Create visualization
volatility_df = pd.DataFrame({
    'Volatility': [f"{v:.0%}" for v in volatilities],
    'Position Size': [f"${s:,.0f}" for s in position_sizes],
    'Portfolio %': [f"{s/portfolio_value:.1%}" for s in position_sizes]
})

print("\nVolatility-Based Position Sizing:")
print(volatility_df.to_string(index=False))
print("\nNote: Higher volatility → Smaller positions (risk management)")

## 5. Circuit Breaker Simulation

Simulate a drawdown scenario that triggers the circuit breaker.

In [ ]:
# Simulate portfolio drawdown over time
np.random.seed(42)

risk_manager = RiskManager(conservative)
initial_value = 100000
peak_value = initial_value

# Simulate daily returns with a downtrend
days = 50
daily_returns = np.random.normal(-0.005, 0.02, days)  # Negative mean (downtrend)
portfolio_values = [initial_value]
drawdowns = [0.0]
circuit_breaker_status = [False]

for ret in daily_returns:
    new_value = portfolio_values[-1] * (1 + ret)
    peak_value = max(peak_value, new_value)
    drawdown = (peak_value - new_value) / peak_value
    
    portfolio_values.append(new_value)
    drawdowns.append(drawdown)
    circuit_breaker_status.append(risk_manager.is_circuit_breaker_active())
    
    # Check if circuit breaker should trigger
    if not risk_manager.is_circuit_breaker_active():
        result = risk_manager.check_portfolio_risk(
            portfolio_value=new_value,
            peak_value=peak_value,
            cash=new_value * 0.25,  # 25% cash
            positions=[]
        )

# Create DataFrame
simulation_df = pd.DataFrame({
    'Day': range(len(portfolio_values)),
    'Portfolio Value': portfolio_values,
    'Drawdown': drawdowns,
    'Circuit Breaker Active': circuit_breaker_status
})

# Find when circuit breaker triggered
cb_trigger_day = simulation_df[simulation_df['Circuit Breaker Active']]['Day'].min()
if pd.notna(cb_trigger_day):
    print(f"\n🚨 Circuit Breaker Triggered on Day {cb_trigger_day}")
    trigger_value = simulation_df.loc[cb_trigger_day, 'Portfolio Value']
    trigger_drawdown = simulation_df.loc[cb_trigger_day, 'Drawdown']
    print(f"   Portfolio Value: ${trigger_value:,.2f}")
    print(f"   Drawdown: {trigger_drawdown:.1%}")
    print(f"   Threshold: {conservative.circuit_breaker_threshold:.1%}")
else:
    print("\n✅ Circuit Breaker Not Triggered")

print(f"\nFinal Portfolio Value: ${portfolio_values[-1]:,.2f}")
print(f"Maximum Drawdown: {max(drawdowns):.1%}")

## 6. Strategy Comparison: No Risk Controls vs. Strict Risk Controls

Compare a simple momentum strategy with and without risk management.

In [ ]:
# Simulate a strategy with and without risk controls
np.random.seed(42)

# Generate mock returns
days = 252  # 1 trading year
daily_returns = np.random.normal(0.0005, 0.02, days)  # Slight positive drift, 20% annual vol

# Add occasional large losses (tail risk)
crash_days = np.random.choice(days, size=3, replace=False)
daily_returns[crash_days] = -0.10  # 10% loss days

# Strategy WITHOUT risk controls
no_risk_values = [100000]
for ret in daily_returns:
    no_risk_values.append(no_risk_values[-1] * (1 + ret))

# Strategy WITH risk controls
risk_manager = RiskManager(conservative)
with_risk_values = [100000]
peak_value = 100000
position_active = True

for ret in daily_returns:
    current_value = with_risk_values[-1]
    peak_value = max(peak_value, current_value)
    
    # Check if we can trade (circuit breaker check)
    if position_active and not risk_manager.is_circuit_breaker_active():
        result = risk_manager.check_portfolio_risk(
            portfolio_value=current_value,
            peak_value=peak_value,
            cash=current_value * 0.25,
            positions=[]
        )
        
        if not result.approved:
            position_active = False
            ret = 0  # No gain/loss if not trading
    else:
        ret = 0  # Circuit breaker active or position closed
    
    with_risk_values.append(current_value * (1 + ret))

# Calculate metrics
def calculate_metrics(values):
    returns = pd.Series(values).pct_change().dropna()
    total_return = (values[-1] - values[0]) / values[0]
    sharpe = returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0
    max_dd = max((max(values[:i+1]) - v) / max(values[:i+1]) for i, v in enumerate(values) if max(values[:i+1]) > 0)
    return total_return, sharpe, max_dd

no_risk_metrics = calculate_metrics(no_risk_values)
with_risk_metrics = calculate_metrics(with_risk_values)

# Display results
comparison_df = pd.DataFrame({
    'Metric': ['Total Return', 'Sharpe Ratio', 'Max Drawdown', 'Final Value'],
    'No Risk Controls': [
        f"{no_risk_metrics[0]:.1%}",
        f"{no_risk_metrics[1]:.2f}",
        f"{no_risk_metrics[2]:.1%}",
        f"${no_risk_values[-1]:,.2f}"
    ],
    'With Risk Controls': [
        f"{with_risk_metrics[0]:.1%}",
        f"{with_risk_metrics[1]:.2f}",
        f"{with_risk_metrics[2]:.1%}",
        f"${with_risk_values[-1]:,.2f}"
    ]
})

print("\nStrategy Comparison:")
print(comparison_df.to_string(index=False))
print("\n📊 Key Takeaway:")
print("Risk controls may reduce returns in good times, but they")
print("preserve capital during crashes and improve risk-adjusted performance.")

## 7. Correlation Filter Example

Demonstrate how correlation checks prevent over-concentrated portfolios.

In [ ]:
# Generate correlated price data
np.random.seed(42)
days = 100

# Market factor (common to all stocks)
market = np.random.randn(days).cumsum()

# Create stocks with varying correlation to market
price_data = pd.DataFrame({
    'AAPL': 100 + market + np.random.randn(days) * 0.5,  # High correlation
    'MSFT': 200 + market * 1.2 + np.random.randn(days) * 0.5,  # High correlation
    'GOOGL': 150 + market * 0.9 + np.random.randn(days) * 0.5,  # High correlation
    'TSLA': 300 + np.random.randn(days).cumsum() * 2,  # Low correlation (independent)
    'GLD': 180 - market * 0.3 + np.random.randn(days) * 0.5,  # Negative correlation
})

# Calculate correlation matrix
corr_matrix = price_data.corr()
print("\nCorrelation Matrix:")
print(corr_matrix.round(2))

# Test adding new positions
risk_manager = RiskManager(conservative)
existing_positions = [{"symbol": "AAPL"}, {"symbol": "MSFT"}]

test_symbols = ["GOOGL", "TSLA", "GLD"]

print("\nCorrelation Check Results:")
print("="*80)
print(f"Existing positions: {[p['symbol'] for p in existing_positions]}")
print(f"Max correlation threshold: {conservative.max_correlation}\n")

for symbol in test_symbols:
    result = risk_manager.check_correlation(
        new_symbol=symbol,
        existing_positions=existing_positions,
        price_data=price_data
    )
    
    status = "✅ APPROVED" if result.approved else "❌ REJECTED"
    print(f"{symbol}: {status}")
    print(f"  Reason: {result.reason}")
    if result.details and 'correlations' in result.details:
        for corr_symbol, corr_value in result.details['correlations']:
            print(f"  Correlation with {corr_symbol}: {corr_value:.2f}")
    print()

## Conclusion

### Key Lessons:

1. **Risk controls preserve capital** during market stress
2. **Circuit breaker prevents catastrophic losses** by halting trading when drawdown threshold is reached
3. **Position limits prevent over-concentration** in single securities
4. **Volatility targeting** adjusts position sizes to maintain consistent risk
5. **Correlation filters** ensure portfolio diversification
6. **Conservative defaults** prioritize capital preservation over maximum returns

### Next Steps:

- Integrate RiskManager into your trading strategies
- Use the Streamlit UI to configure risk parameters
- Backtest your strategies with different risk profiles
- Monitor the breach log to understand when and why limits are hit

### Remember:

**The goal is not to maximize returns, but to survive and thrive over the long term.**

Risk management is the difference between a sustainable trading operation and a blown-up account.